<a href="https://colab.research.google.com/github/hrbae/LG_AI_training/blob/main/M2_System%20Optimization%20by%20ML/LG_Day2_Reinforcement%20Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 과제. 작업자 배정 문제 - 추가 제약사항 

- 목적: 총 비용 최소화
- 제약조건 1: 각 작업자를 최소 1개, 최대 3개 작업에 할당 (기존) -> 각 작업자를 최소 2개, 최대 3개 작업에 할당 (변경)
- 제약조건 2: 각 작업은 최소 1명의 작업자에게만 할당이 되어야 함   
- 제약조건 3: 작업자는 2개의 팀으로 분할되며 (기존) -> 작업자는 3개의 팀으로 분할되며 (변경), 각 팀은 최소 2개 이상 최대 4개까지 작업할당이 가능

| Worker | Team   | Task 0 | Task 1 | Task 2 | Task 3 | Task 4 | Task 5 | Task 6 | Task 7 | Task 8 | Task 9 | Task 10 |
| ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------  |
| 0      | Team 1 | 90     | 80     | 75     | 100    | 100    | 140    | 120    | 35     | 210    | 130    | 80      |
| 1      | Team 1 | 65     | 35     | 55     | 165    | 120    | 55     | 40     | 45     | 180    | 220    | 130     |
| 2      | Team 2 | 225    | 45     | 90     | 55     | 140    | 100    | 60     | 15     | 140    | 100    | 20      |
| 3      | Team 2 | 35     | 20     | 95     | 315    | 50     | 80     | 80     | 35     | 100    | 60     | 90      |
| 4      | Team 3 | 10     | 150    | 90     | 120    | 40     | 200    | 90     | 95     | 300    | 50     | 100     |
| 5      | Team 3 | 80     | 40     | 100    | 30     | 70     | 150    | 30     | 15     | 400    | 90     | 150     |

##### 기존 코드

In [2]:
import numpy as np
import random
from collections import defaultdict
import copy

In [3]:
class taskAllocation:
    
    def __init__(self, epsilon, discount_factor, learning_rate, constraints):
        
        self.epsilon = epsilon
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate
        
        self.Action_Index = [0, 1, 2, 3, 4, 5]
        self.Action = [0, 1, 2, 3, 4, 5]
        
        self.team1 , self.team2 , self.team_min , self.team_max , self.w_minimum , self.w_maximum  = constraints
        self.QTable = defaultdict(lambda : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        
        
    def arg_max_Moving(self,state_Moving):
        
        max_index_list = []
        max_value = state_Moving[0]
        
        for index, value in enumerate(state_Moving):
            
            if value > max_value:
                
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
                
            elif value == max_value:
                
                max_index_list.append(index)
                
        return random.choice(max_index_list)
    
    
    def Get_Action(self, state):
        
        if np.random.rand() < self.epsilon:    
            Action = np.random.choice(self.Action_Index)
            
        else:
            state_str = [str(x) for x in state]
            state_str2 = '.'.join(state_str)
            '''
            state_Moving = Allocation.QTable[state_str2]
            '''
            state_Moving = self.QTable[state_str2]
            Action = self.arg_max_Moving(state_Moving = state_Moving)
            
        return Action
    
    def Q_learning(self, state, next_state, reward):
        
        Action = next_state[-1]
        state_str = [str(x) for x in state]
        state_str2 = '.'.join(state_str)
        Q1 = self.QTable[state_str2][Action]
        
        nextstate_str = [str(x) for x in next_state]
        nextstate_str2 = '.'.join(nextstate_str)
        Q2 = reward + self.discount_factor * max(self.QTable[nextstate_str2])
        
        self.QTable[state_str2][Action] += self.learning_rate*(Q2-Q1)
        
    def Get_State(self, x, costs, steps, Action, team1, team2):
        
        ## Constraint 1
        if sum(x[Action,:]) < self.w_minimum:
            penalty1_1 = -10
        else:
            penalty1_1 = 0
        if sum(x[Action,:]) > self.w_maximum:
            penalty1_2 = -10
        else:
            penalty1_2 = 0
    
        
        ##  Constraint 2
        if sum(x[team1,:].sum(axis=1)) < self.team_min:
            penalty3_1 = -10
        else:
            penalty3_1 = 0
        if sum(x[team1,:].sum(axis=1)) > self.team_max:
            penalty3_2 = -10
        else:
            penalty3_2 = 0
        if sum(x[team2,:].sum(axis=1)) < self.team_min:
            penalty3_3 = -10
        else:
            penalty3_3 = 0
        if sum(x[team2,:].sum(axis=1)) > self.team_max:
            penalty3_4 = -10
        else:
            penalty3_4 = 0
            
        cost = costs[Action][steps] * -1
        reward = penalty1_1 + penalty1_2 + penalty3_1 + penalty3_2 + penalty3_3 + penalty3_4 + cost
        
        return reward

In [4]:
team1 = "변경된 상황에 맞게 구현하시오"
team2 = "변경된 상황에 맞게 구현하시오"
team_min = "변경된 상황에 맞게 구현하시오"
team_max = "변경된 상황에 맞게 구현하시오"
w_minimum = "변경된 상황에 맞게 구현하시오"
w_maximum = "변경된 상황에 맞게 구현하시오"

constraints = []
constraints.append(team1)
constraints.append(team2)
constraints.append(team_min)
constraints.append(team_max)
constraints.append(w_minimum)
constraints.append(w_maximum)

costs = [
    [90, 80, 75, 100, 100, 140, 120, 35, 210, 130, 80],
    [65, 35, 55, 165, 120, 55, 40, 45, 180, 220, 130],
    [225, 45, 90, 55, 140, 100, 60, 15, 140, 100, 20],
    [35, 20, 95, 315, 50, 80, 80, 35, 100, 60, 90],
    [10, 150, 90, 120, 40, 200, 90, 95, 300, 50, 100],
    [80, 40, 100, 30, 70, 150, 30, 15, 400, 90, 150]
]

num_workers = 6
num_tasks = 11

In [5]:
epsilon = 0.2
discount_factor = 0.9
learning_rate = 0.2

Allocation = taskAllocation(epsilon, discount_factor, learning_rate, constraints)

In [6]:
E = []
rewardList = []
costList = []

num_episode = 50000
for e in range(num_episode):
    
    state = []
    x = np.zeros((num_workers, num_tasks))
        
    total_reward = 0 
    steps = 0
    total_cost = 0
    
    while True:
    
        Action = Allocation.Get_Action(state)
        x[Action, steps] += 1
        
        reward = Allocation.Get_State(x, costs, steps, Action, team1, team2)
        next_state = copy.deepcopy(state)
        next_state.append(Action)
        Allocation.Q_learning(state, next_state, reward)
        
        total_reward += reward
        state = copy.deepcopy(next_state)
        total_cost += costs[Action][steps]
        steps += 1

        
        if steps == num_tasks:
            costList.append(total_cost)
            E.append(e)
            rewardList.append(total_reward)
            
            break

TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
import pylab
pylab.plot(E, rewardList,'b')